In [1]:
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
from scipy import stats
import os
import re
import json
import presto  # import prestosql python client
import psycopg2
import warnings

import altair as alt
from altair import datum
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import datetime
warnings.simplefilter('ignore', InsecureRequestWarning)
from matplotlib import pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None
alt.data_transformers.disable_max_rows()
pd.options.display.max_rows = 4000

### Analysis for LPC and ENC quantitation, plot the distribution on heatmap

In [2]:
pq_data = pd.read_csv('axygen_seal_study_pq_run_data_all.csv')
pq_data_final = pq_data[pq_data.data_category == 'final']

pq_data_final['en_row'] = pq_data_final['well'].str[0]
pq_data_final['en_column'] = pq_data_final['well'].str[1:].astype('int')
pq_data_final['label'] = round(pq_data_final['quant_value'], 2).astype('str')
pq_data_final.head()

/home/jiwang/miniconda3/envs/my_environment/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/jiwang/miniconda3/envs/my_environment/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/jiwang/miniconda3/envs/my_environment/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,sealing_method,sta_id,quant_category,test,data_category,well,quant_value,en_row,en_column,label
0,ALPS,STA00005,LPC quant,original run,final,A1,11.0,A,1,11.0
1,ALPS,STA00005,LPC quant,original run,final,B1,10.9,B,1,10.9
2,ALPS,STA00005,LPC quant,original run,final,C1,11.0,C,1,11.0
3,ALPS,STA00005,LPC quant,original run,final,D1,11.2,D,1,11.2
4,ALPS,STA00005,LPC quant,original run,final,E1,11.5,E,1,11.5


In [3]:
heatmap = alt.Chart(pq_data_final).mark_rect().encode(
    x=alt.X('en_column:O', 
            title='Column', 
            axis=alt.Axis(labelAngle=0)
           ),
    y=alt.Y('en_row:O', 
            title='Row'
           ),
    color=alt.Color('quant_value:Q', scale=alt.Scale(scheme='inferno', domainMid=0.31))
    
).properties(
    width=400,
    height=300
)

text = alt.Chart(pq_data_final).mark_text( baseline='middle', color='black', size=12, lineBreak='\n').encode(
    x=alt.X('en_column:O', 
            title='Column', 
            axis=alt.Axis(labelAngle=0)
           ),
    y=alt.Y('en_row:O', 
            title='Row'
           ),
    text=alt.Text('label:N')
)

alt.layer(heatmap, text, data= pq_data_final, width=600, height = 500).facet(
    row = alt.Row('sealing_method:N'), 
    column = alt.Column('quant_category:N', sort = ['LPC quant ', 'ENC quant']),
    title=alt.TitleParams(f'Heatmap of LPC Quant and ENC Quant of STAR PQ Test 1 Run',
                          anchor='middle',
                          fontSize=15,
                          dy=-10)
).resolve_scale(
     color = 'independent'
).resolve_axis(
    x='independent', y='independent'
).configure_axis(
    titleFontSize=14,
    labelFontSize=12
).configure_legend(
    titleFontSize=14,
    labelFontSize=14
).configure_facet(
    spacing=20
).configure_header(
    titleFontSize=14,
   labelFontSize=14
)

alt.FacetChart(...)

### Analysis for cross-contamination rate, plot the result on heatmap

In [4]:
cc_data = pd.read_csv('cross_contamination_study_data.csv')
cc_data['row'] = cc_data['spark_plate_wellid'].str[0]
cc_data['column'] = cc_data['spark_plate_wellid'].str[1:].astype('int')

# data_sub_en_fil.head()
cc_data['label'] =  cc_data['Test'].astype('str') + '\n' + cc_data['value'].astype('str')
cc_data['status'] = cc_data[['value', 'column']].apply(lambda x: 'PASS' if x['value'] <= 700 or x['column'] >= 11 else 'FAIL', axis = 1)
cc_data['spark_plate_title'] = 'Fluorescence reading value of ' +  cc_data['spark_plate'] 
cc_data.head()

,study_type,spark_plate,Test,sealing_vortexing,spark_plate_wellid,value,row,column,label,status,spark_plate_title
0,crosscontamination,plate1_test1&2,test1,ALPS+VWR,A1,0,A,1,test1\n0,PASS,Fluorescence reading value of plate1_test1&2
1,crosscontamination,plate1_test1&2,test1,ALPS+VWR,B1,0,B,1,test1\n0,PASS,Fluorescence reading value of plate1_test1&2
2,crosscontamination,plate1_test1&2,test1,ALPS+VWR,C1,0,C,1,test1\n0,PASS,Fluorescence reading value of plate1_test1&2
3,crosscontamination,plate1_test1&2,test1,ALPS+VWR,D1,0,D,1,test1\n0,PASS,Fluorescence reading value of plate1_test1&2
4,crosscontamination,plate1_test1&2,test1,ALPS+VWR,E1,0,E,1,test1\n0,PASS,Fluorescence reading value of plate1_test1&2


In [5]:
domain = ['PASS', 'FAIL']
range_ = ['black', 'blue']


heatmap = alt.Chart(cc_data).mark_rect().encode(
    x=alt.X('column:O', 
            title='Column', 
            axis=alt.Axis(labelAngle=0)
           ),
    y=alt.Y('row:O', 
            title='Row'
           ),
    color=alt.Color('value:Q', scale=alt.Scale(scheme='lightorange', domainMid=700))
    
).properties(
    width=600,
    height=300
)

text = alt.Chart(cc_data).mark_text(align='center', baseline='middle',  color='black', size=13, lineBreak='\n' ).encode(
    x=alt.X('column:O', 
            title='Column'
           ),
    y=alt.Y('row:O', 
            title='Row'
           ),
    color = alt.Color('status:N', scale=alt.Scale(domain=domain, range=range_)),
    text=alt.Text('label:N')
)


chart1 = alt.layer(heatmap, text, data=cc_data, width=800, height = 400).facet(
    facet=alt.Facet('spark_plate_title:N'), columns=1,
).properties(
    title=alt.TitleParams(f'Cross-contamination study result',
                          anchor='middle',
                          fontSize=15,
                          dy=-10)
).resolve_scale( 
      x='independent', y='independent'
#     color='independent'
).resolve_axis(
    x='independent', y= 'independent'
).configure_axis(
    titleFontSize=16,
    labelFontSize=14
).configure_legend(
    titleFontSize=14,
    labelFontSize=14
).configure_facet(
    spacing=20
).configure_header(
    titleFontSize=16,
    labelFontSize=16
)
chart1

alt.FacetChart(...)

### Plot the standard Curve for each test

In [6]:
cc_std_conc = {'standard': ['std1', 'std2', 'std3', 'std4', 'std5', 'std6', 'std7', 'std8'],
                      'fluorescein_concentration': [0.2, 0.1,0.05,  0.025, 0.0125, 0.00625, 0.003125, 0.0003906]}
cc_std_conc_df = pd.DataFrame(cc_std_conc)

In [7]:
plate_sel = 'plate1_test1&2'
cc_data_sub = cc_data[(cc_data.spark_plate == plate_sel) & (cc_data.Test.str.startswith('std'))]
cc_data_sub_sum = cc_data_sub.groupby(['Test'])['value'].mean().reset_index().rename(columns = {'Test':'standard', 'value':'fluorescence_reading'})
cc_data_sub_sum = cc_data_sub_sum[~cc_data_sub_sum.standard.isin(['std1', 'std2'])].merge(cc_std_conc_df, on = 'standard')

# Perform linear regression using numpy
slope, intercept = np.polyfit(cc_data_sub_sum['fluorescein_concentration'], cc_data_sub_sum['fluorescence_reading'], 1)


# Calculate R² value
predicted_response = slope * cc_data_sub_sum['fluorescein_concentration'] + intercept
ss_res = np.sum((cc_data_sub_sum['fluorescence_reading'] - predicted_response) ** 2)
ss_tot = np.sum((cc_data_sub_sum['fluorescence_reading'] - np.mean(cc_data_sub_sum['fluorescence_reading'])) ** 2)
r_squared = 1 - (ss_res / ss_tot)

scatter_plot = alt.Chart(cc_data_sub_sum).mark_point().encode(
    x=alt.X('fluorescein_concentration:Q', title='Concentration of Fluorescein'),
    y=alt.Y('fluorescence_reading:Q', title='Fluorescence Reading')
)


regression_data = pd.DataFrame({
    'fluorescein_concentration': cc_data_sub_sum['fluorescein_concentration'],
    'fluorescence_reading': slope * cc_data_sub_sum['fluorescein_concentration'] + intercept
})


# Add a regression line
regression_line = alt.Chart(regression_data).mark_line(color='blue', strokeDash=[5, 5]).encode(
    x='fluorescein_concentration:Q',
    y= 'fluorescence_reading:Q'
)


# Create text for the regression equation
equation_text = alt.Chart(pd.DataFrame({
#     'text': [f'y = {slope:.2f}x + {intercept:.2f}'],
    'text': [f'y = {slope:.2f}x + {intercept:.2f}\nR² = {r_squared:.2f}'],
    'x': [cc_data_sub_sum['fluorescein_concentration'].min()],
    'y': [slope * cc_data_sub_sum['fluorescein_concentration'].max() + intercept]
})).mark_text(
    align='left', baseline='top', dx=5, dy= -5, fontSize=16, color='black'
).encode(
    x='x:Q',
    y='y:Q',
    text='text:N'
)

# equation_text

# Combine the scatter plot and regression line
standard_curve1 = alt.layer(scatter_plot, regression_line, equation_text).properties(
    width=600,
    height=300,
    title=f'Standard Curve of {plate_sel}'
).configure_axis(
    titleFontSize=16,
    labelFontSize=14
).configure_title(
    fontSize=16
)

# Display the chart
standard_curve1

alt.LayerChart(...)

In [8]:
plate_sel = 'plate2_test3&4'
cc_data_sub = cc_data[(cc_data.spark_plate == plate_sel) & (cc_data.Test.str.startswith('std'))]
cc_data_sub_sum = cc_data_sub.groupby(['Test'])['value'].mean().reset_index().rename(columns = {'Test':'standard', 'value':'fluorescence_reading'})
cc_data_sub_sum = cc_data_sub_sum[~cc_data_sub_sum.standard.isin(['std1', 'std2'])].merge(cc_std_conc_df, on = 'standard')
# Perform linear regression using numpy
slope, intercept = np.polyfit(cc_data_sub_sum['fluorescein_concentration'], cc_data_sub_sum['fluorescence_reading'], 1)


# Calculate R² value
predicted_response = slope * cc_data_sub_sum['fluorescein_concentration'] + intercept
ss_res = np.sum((cc_data_sub_sum['fluorescence_reading'] - predicted_response) ** 2)
ss_tot = np.sum((cc_data_sub_sum['fluorescence_reading'] - np.mean(cc_data_sub_sum['fluorescence_reading'])) ** 2)
r_squared = 1 - (ss_res / ss_tot)

scatter_plot = alt.Chart(cc_data_sub_sum).mark_point().encode(
    x=alt.X('fluorescein_concentration:Q', title='Concentration of Fluorescein'),
    y=alt.Y('fluorescence_reading:Q', title='Fluorescence Reading')
)


regression_data = pd.DataFrame({
    'fluorescein_concentration': cc_data_sub_sum['fluorescein_concentration'],
    'fluorescence_reading': slope * cc_data_sub_sum['fluorescein_concentration'] + intercept
})


# Add a regression line
regression_line = alt.Chart(regression_data).mark_line(color='blue', strokeDash=[5, 5]).encode(
    x='fluorescein_concentration:Q',
    y= 'fluorescence_reading:Q'
)


# Create text for the regression equation
equation_text = alt.Chart(pd.DataFrame({
#     'text': [f'y = {slope:.2f}x + {intercept:.2f}'],
    'text': [f'y = {slope:.2f}x + {intercept:.2f}\nR² = {r_squared:.2f}'],
    'x': [cc_data_sub_sum['fluorescein_concentration'].min()],
    'y': [slope * cc_data_sub_sum['fluorescein_concentration'].max() + intercept]
})).mark_text(
    align='left', baseline='middle', dx=5, dy=-5, fontSize=16, color='black'
).encode(
    x='x:Q',
    y='y:Q',
    text='text:N'
)

# equation_text


# Combine the scatter plot and regression line
standard_curve2 = alt.layer(scatter_plot, regression_line, equation_text).properties(
    width=600,
    height=300,
    title=f'Standard Curve of {plate_sel}'
).configure_axis(
    titleFontSize=14,
    labelFontSize=12
).configure_title(
    fontSize=16
)

# Display the chart
standard_curve2

alt.LayerChart(...)

In [11]:
plate_sel = 'plate3_test5&6'
cc_data_sub = cc_data[(cc_data.spark_plate == plate_sel) & (cc_data.Test.str.startswith('std'))]
cc_data_sub_sum = cc_data_sub.groupby(['Test'])['value'].mean().reset_index().rename(columns = {'Test':'standard', 'value':'fluorescence_reading'})
cc_data_sub_sum = cc_data_sub_sum[~cc_data_sub_sum.standard.isin(['std1', 'std2'])].merge(cc_std_conc_df, on = 'standard')
# Perform linear regression using numpy
slope, intercept = np.polyfit(cc_data_sub_sum['fluorescein_concentration'], cc_data_sub_sum['fluorescence_reading'], 1)


# Calculate R² value
predicted_response = slope * cc_data_sub_sum['fluorescein_concentration'] + intercept
ss_res = np.sum((cc_data_sub_sum['fluorescence_reading'] - predicted_response) ** 2)
ss_tot = np.sum((cc_data_sub_sum['fluorescence_reading'] - np.mean(cc_data_sub_sum['fluorescence_reading'])) ** 2)
r_squared = 1 - (ss_res / ss_tot)

scatter_plot = alt.Chart(cc_data_sub_sum).mark_point().encode(
    x=alt.X('fluorescein_concentration:Q', title='Concentration of Fluorescein'),
    y=alt.Y('fluorescence_reading:Q', title='Fluorescence Reading')
)


regression_data = pd.DataFrame({
    'fluorescein_concentration': cc_data_sub_sum['fluorescein_concentration'],
    'fluorescence_reading': slope * cc_data_sub_sum['fluorescein_concentration'] + intercept
})


# Add a regression line
regression_line = alt.Chart(regression_data).mark_line(color='blue', strokeDash=[5, 5]).encode(
    x='fluorescein_concentration:Q',
    y= 'fluorescence_reading:Q'
)


# Create text for the regression equation
equation_text = alt.Chart(pd.DataFrame({
#     'text': [f'y = {slope:.2f}x + {intercept:.2f}'],
    'text': [f'y = {slope:.2f}x + {intercept:.2f}\nR² = {r_squared:.2f}'],
    'x': [cc_data_sub_sum['fluorescein_concentration'].min()],
    'y': [slope * cc_data_sub_sum['fluorescein_concentration'].max() + intercept]
})).mark_text(
    align='left', baseline='middle', dx=5, dy=-5, fontSize=16, color='black'
).encode(
    x='x:Q',
    y='y:Q',
    text='text:N'
)

# equation_text


# Combine the scatter plot and regression line
standard_curve3 = alt.layer(scatter_plot, regression_line, equation_text).properties(
    width=600,
    height=300,
    title=f'Standard Curve of {plate_sel}'
).configure_axis(
    titleFontSize=16,
    labelFontSize=14
).configure_title(
    fontSize=16
)

# Display the chart
standard_curve3

alt.LayerChart(...)

In [12]:
plate_sel = 'plate4_test7&8'
cc_data_sub = cc_data[(cc_data.spark_plate == plate_sel) & (cc_data.Test.str.startswith('std'))]
cc_data_sub_sum = cc_data_sub.groupby(['Test'])['value'].mean().reset_index().rename(columns = {'Test':'standard', 'value':'fluorescence_reading'})
cc_data_sub_sum = cc_data_sub_sum[~cc_data_sub_sum.standard.isin(['std1', 'std2'])].merge(cc_std_conc_df, on = 'standard')

# Perform linear regression using numpy
slope, intercept = np.polyfit(cc_data_sub_sum['fluorescein_concentration'], cc_data_sub_sum['fluorescence_reading'], 1)


# Calculate R² value
predicted_response = slope * cc_data_sub_sum['fluorescein_concentration'] + intercept
ss_res = np.sum((cc_data_sub_sum['fluorescence_reading'] - predicted_response) ** 2)
ss_tot = np.sum((cc_data_sub_sum['fluorescence_reading'] - np.mean(cc_data_sub_sum['fluorescence_reading'])) ** 2)
r_squared = 1 - (ss_res / ss_tot)

scatter_plot = alt.Chart(cc_data_sub_sum).mark_point().encode(
    x=alt.X('fluorescein_concentration:Q', title='Concentration of Fluorescein'),
    y=alt.Y('fluorescence_reading:Q', title='Fluorescence Reading')
)


regression_data = pd.DataFrame({
    'fluorescein_concentration': cc_data_sub_sum['fluorescein_concentration'],
    'fluorescence_reading': slope * cc_data_sub_sum['fluorescein_concentration'] + intercept
})


# Add a regression line
regression_line = alt.Chart(regression_data).mark_line(color='blue', strokeDash=[5, 5]).encode(
    x='fluorescein_concentration:Q',
    y= 'fluorescence_reading:Q'
)


# Create text for the regression equation
equation_text = alt.Chart(pd.DataFrame({
#     'text': [f'y = {slope:.2f}x + {intercept:.2f}'],
    'text': [f'y = {slope:.2f}x + {intercept:.2f}\nR² = {r_squared:.2f}'],
    'x': [cc_data_sub_sum['fluorescein_concentration'].min()],
    'y': [slope * cc_data_sub_sum['fluorescein_concentration'].max() + intercept]
})).mark_text(
    align='left', baseline='middle', dx=5, dy=-5, fontSize=16, color='black'
).encode(
    x='x:Q',
    y='y:Q',
    text='text:N'
)

# equation_text

# Combine the scatter plot and regression line
standard_curve4 = alt.layer(scatter_plot, regression_line, equation_text).properties(
    width=600,
    height=300,
    title=f'Standard Curve of {plate_sel}'
).configure_axis(
    titleFontSize=16,
    labelFontSize=14
).configure_title(
    fontSize=16
)

# Display the chart
standard_curve4

alt.LayerChart(...)